Name: run_wc_dataset
Version: 2.0
Author: Anish Vasan
Organization: Chen/Eyckmans Lab, Boston University
Date: 20230403

In [ ]:
#imports
import os, shutil, fnmatch, tkinter
from tkinter.filedialog import askdirectory
from pathlib import Path
from woundcompute import image_analysis as ia
import time

In [ ]:
def define_basename_list():

basename_list = []
for file in os.listdir(path):
    if file.endswith('.nd'):
        name, ext = os.path.splitext(file)
        basename_list.append(name)
        #create folders for each expt
        os.makedirs(path_output + '/' + name)
    return(basename_list)